In [80]:
import numpy as np
import scipy.misc
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import scipy.misc
import os
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [81]:
cascPath = "haarcascade_frontalface_default.xml"
# Create the haar cascade
faceCascade = cv2.CascadeClassifier(cascPath)


In [112]:
## read dataset1
full_face = []
full_labels = []
all_images = os.listdir('yalefaces')
for j in range(1,16):    # there are 15 different people in the dataset
    img_list = [filename for filename in all_images if filename.startswith('subject%.2d'%j)]
#     img_list = glob.glob('yalefaces/subject%.2d*'%j)
    face1 = np.zeros((len(img_list),64,64))
    for k,img in enumerate(img_list):
        I = scipy.misc.imread(os.path.join('yalefaces',img))
        I = cv2.fastNlMeansDenoising(I, None, 9, 13)
        I = cv2.GaussianBlur(I, (5,5), 0)
        I = cv2.equalizeHist(I)
        faces = faceCascade.detectMultiScale(
            I,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30))
        try : 
            x,y,h,w = faces[0]
            face1[k,...] = zoom(I[y:y+h,x:x+w],[64./h,64./w])     
        except : 
            x,y,h,w=[0,0,479,639]
            face1[k,...] = zoom(I[y:y+h,x:x+w],[64./h,64./w])   
        
    labels = (j-1)*np.ones((11,))
    full_face.append(face1)
    full_labels.append(labels)
full_face = np.stack(full_face)    # contains the images of all faces
full_labels = np.stack(full_labels)   # contains the ID of all the faces


In [113]:
full_face = full_face/255 #normalise data
print(full_face.shape)
print(full_labels.shape)

(15, 11, 64, 64)
(15, 11)


In [114]:
#similar 1 dissimilar 0
# 2 images of same and different people
similarity=[]
labels = []
pairs = []
sim=0
nonsim=0
for i in range(0,40000):
        b = np.random.randint(0,15,2)
        a = np.random.randint(0,11,2)
        
        
        if(b[0]==b[1]):
            similarity.append(1)
            labels.append(b)
            pairs.append(a)
            sim+=1
        else:
            if nonsim>sim : 
                continue
            else : 
                similarity.append(0)
                labels.append(b)
                pairs.append(a)
                nonsim+=1

In [115]:
print(len(similarity),sum(similarity))

5371 2685


In [116]:
trainX = np.zeros((len(similarity),64,64))

In [117]:
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances

In [118]:
X_train = np.zeros((len(similarity),64*64))
for i in range(len(similarity)):
    a = labels[i]
    b = pairs[i]
    face1 = full_face[a[0],b[0],:,:]
    face2 = full_face[a[1],b[1],:,:]
    face = face1-face2
    face= euclidean_distances(face,face)#np.multiply(face,face)#abs(np.multiply(face1,face1)-np.multiply(face2,face2))
    #face=cosine_similarity(face3,face3)
    trainX[i,:,:]=face
    X_train[i,:]=trainX[i,:,:].ravel()

In [119]:
face

array([[0.        , 0.29382992, 0.5791189 , ..., 3.0169609 , 2.85484271,
        2.95690695],
       [0.29382992, 0.        , 0.32142512, ..., 2.9129284 , 2.75516362,
        2.85586334],
       [0.5791189 , 0.32142512, 0.        , ..., 2.85617565, 2.70936406,
        2.8077199 ],
       ...,
       [3.0169609 , 2.9129284 , 2.85617565, ..., 0.        , 0.5631378 ,
        0.54854321],
       [2.85484271, 2.75516362, 2.70936406, ..., 0.5631378 , 0.        ,
        0.26484383],
       [2.95690695, 2.85586334, 2.8077199 , ..., 0.54854321, 0.26484383,
        0.        ]])

In [120]:
X_train.shape

(5371, 4096)

In [121]:
xtrain1,xtest1,ytrain,ytest=train_test_split(X_train, similarity,test_size=0.3)

In [91]:
xtrain1.shape

(3684, 4096)

In [58]:
# if PCA
from sklearn.decomposition import PCA
pca = PCA(whiten=True)
pca.fit(xtrain1)
n_components = np.argmin(np.cumsum(pca.explained_variance_ratio_)<=0.99)
print(n_components)
xtrain=pca.transform(xtrain1)
xtrain=xtrain[:,:n_components]
xtest=pca.transform(xtest1)
xtest=xtest[:,:n_components]

378


In [149]:
# if LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda=LinearDiscriminantAnalysis()
lda.fit(xtrain1,ytrain)
xtrain=lda.transform(xtrain1)
xtest=lda.transform(xtest1)
print(xtrain.shape,xtest.shape)

(2009, 1) (670, 1)


In [122]:
# if nothing
xtrain=xtrain1
xtest=xtest1
xtrain.shape

(3759, 4096)

In [15]:
from sklearn.metrics import f1_score

In [59]:
from sklearn.svm import SVC
clf=SVC(C=30.0,kernel='rbf')
clf.fit(xtrain,ytrain)
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  1.0
Test Accuracy :  0.9817204301075269
Train F1 Score :  1.0
Test F1 Score :  0.9816810344827586


In [69]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2000,1000,10), random_state=1)
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(1), random_state=1)
clf.fit(xtrain,ytrain)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(2000, 1000, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [66]:
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  1.0
Test Accuracy :  0.9779569892473118
Train F1 Score :  1.0
Test F1 Score :  0.978272390037096


In [134]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(C=50)
clf.fit(xtrain,ytrain)
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  1.0
Test Accuracy :  0.9708436724565757
Train F1 Score :  1.0
Test F1 Score :  0.9708617482951023


In [84]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10, max_depth=7,random_state=7)
clf.fit(xtrain,ytrain)
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  0.941193853427896
Test Accuracy :  0.9040043883708173
Train F1 Score :  0.9418638621092609
Test F1 Score :  0.9065670048051255


In [91]:
from sklearn.ensemble import BaggingClassifier
clf=BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',max_depth=40),n_estimators=100)
clf.fit(xtrain,ytrain)
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  1.0
Test Accuracy :  0.9687328579264948
Train F1 Score :  1.0
Test F1 Score :  0.9692722371967655


In [123]:
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier(criterion='entropy',max_depth=40)#,max_features='sqrt')
clf.fit(xtrain,ytrain)
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  1.0
Test Accuracy :  0.9317617866004962
Train F1 Score :  1.0
Test F1 Score :  0.9325153374233129


In [124]:
import h5py
h5f = h5py.File('test.h5','r')
a = list(h5f.keys())
## read the dataset
full_face = []
full_labels = []
img_list=[h5f[i] for i in a]
tface1 = np.zeros((len(img_list),64,64))
for k,img in enumerate(img_list):
    I = np.array(img)#scipy.misc.imread(os.path.join('yalefaces',img))
    #print(type(I))
    I = cv2.fastNlMeansDenoising(I, None, 9, 13)
    I = cv2.GaussianBlur(I, (5,5), 0)
    I = cv2.equalizeHist(I)
    faces = faceCascade.detectMultiScale(
        I,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30))
    #print(k)
    #print(faces)
    try : 
        x,y,h,w = faces[0]
        tface1[k,...] = zoom(I[y:y+h,x:x+w],[64./h,64./w])
    except : 
        x,y,h,w=[0,0,111,91]
        tface1[k,...] = zoom(I[y:y+h,x:x+w],[64./h,64./w])

In [125]:
tface1 = tface1/255

In [126]:
test_pairs = np.genfromtxt('image_pairs.csv', dtype=None, delimiter=',', names=True)

In [135]:
print(clf)
tface1.shape

LogisticRegression(C=50, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


(120, 64, 64)

In [136]:
euc=[]

for i in range(3540):#(3540):
    x1 = np.zeros((64,64))
    x0 = np.zeros((64,64))
    ids = test_pairs[i][0]
    image1=test_pairs[i][1]
    image2=test_pairs[i][2]
    x0 = tface1[a.index(str(test_pairs[i][1])),:,:]
    x1 = tface1[a.index(str(test_pairs[i][2])),:,:]
    if(i%1000==0):
        print(i)
    face = x0-x1
    face =euclidean_distances(face,face)# np.multiply(face,face)#abs(np.multiply(face1,face1)-np.multiply(face2,face2))#cosine_similarity(face1,face2)
    #face=cosine_similarity(face3,face3)
    face=face.ravel()
    inp = np.zeros((1,64*64))
    inp[0,:]=face
    #inp=pca.transform(inp)
    #inp=inp[:,:n_components]
    euc.append(clf.predict(inp))

0
1000
2000
3000


In [138]:
sum(euc)

array([328])

In [139]:
#1 is similar; 0 is different
with open('to_submit/lr_c50_nopca.csv','w') as f : 
    f.write('Id,Expected\n')
    for i in range(len(test_pairs)) : 
        ids = test_pairs[i][0]
        temp = euc[i][0]
        f.write(str(ids)+','+str(temp)+'\n')

In [ ]:
euc = np.asarray(euc)

In [68]:
sum(clf.predict(X_train))

1367